# **Common Prosperity Simulation**
## Computational Physics Lab 
Author: Yuan-Yen Peng, Chih-Tian Ho  
E-mail: garyphys0915@gapp.nthu.edu.tw, chihtian.ho@gapp.nthu.edu.tw  
Dept of Physics, NTHU, Taiwan   
Date: 
Version: 1.0  
License: MIT

In [6]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import matplotlib.animation as animation
from pathlib import Path
from numba import jit, njit, prange, set_num_threads
import glob

### initial conditions

In [7]:
# initial setup
problem_name = 'Common_Prosperity'
io_screen = True
io_freq = 10 # how many step to record the data
io_title = problem_name

# initial conditions
width, length = 100, 100
N = int(width * length)
J = 1 # energy constant


### Grid generator

In [8]:
def grid_generator(width, length, init_rank:float = 3.0, dec: int = 2):
  '''
  This is a grid generator
  :para width: width of the grid
  :para length: lerngth of the grid
  :para init_rank: bounds of initial rank of the grid; i.e., if use 3, --> [-3, 3]
  :para dec: decimal for the function <round>
  '''

  grid_positive = np.round(init_rank * np.random.random(int(N/2)), dec)
  grid_negative = np.round(-init_rank * np.random.random(int(N/2)), dec)
  grid = np.append(grid_positive, grid_negative)
  np.random.shuffle(grid) # set it to random
  grid = grid.reshape((width, length))

  # add the boundaries to four margins with minimum initial rank - 1
  tor = - (init_rank + 1)
  res = np.array([tor * np.ones(length)])
  grid_boundary = np.concatenate((res, grid, res), axis=0)
  grid_boundary = np.insert(grid_boundary, (0, length), tor, axis=1)

  return grid, grid_boundary

In [9]:
def plot(arr, header, size = 8, dpi = 150):
  '''
  :para arr: input data: 2D Array
  '''

  figure(figsize=(size, size), dpi=dpi)
  plt.imshow(arr, cmap='coolwarm', extent=[-width/2,width/2,-length/2,length/2], alpha=.85)
  plt.colorbar()
  plt.title(f'{header}')

  return

### output

make the directories

In [10]:
io_folder = "data_" + io_title
Path(io_folder).mkdir(parents=True, exist_ok=True)
io_folder_fig = "fig_" + io_title
Path(io_folder_fig).mkdir(parents=True, exist_ok=True)

output figures

In [11]:
def output_fig(n, arr, time):
  """
  Write simulation data into a file named "fig"
  """
  header = "Time = {}".format(time)
  plot(arr, header)
  fig = f'{io_folder_fig}/fig_{io_title}_{str(n).zfill(3)}.png'
  plt.savefig(fig)
  return 

In [12]:
# # test

# arr = grid_generator(width, length, init_rank = 3.0, dec = 2)
# output_fig(12, arr[0], 23)
# print(arr[1])

output info files

In [13]:
def output(tag, spin, energy, fn, time):
  """
  Write simulation data into a file named "fn"
  """
  header = """
          ----------------------------------------------------
          Data from a hierachy simulation. 

          rows are ith gird's "seat" of playground (this grid); 
          columns are :tag, spin, x, y, site's energy

          ----------------------------------------------------
          """
  header += "Time = {}".format(time)
  np.savetxt(fn,(tag, 
                 spin, 
                 energy),header=header)
  return

### main exchange functions

In [69]:
# setup

n = 1000 # time steps
tags = np.arange(0, N).reshape((width, length)) # label of position
spin = grid_generator(width, length, init_rank = 3.0, dec = 2)[1] # include boundary
tmax = 100 # [code unit]
dt = 0.1 # [code unit]

# set the standard diviation for x & y are both 3 
sigma_x = 100.0
sigma_y = 100.0

# set eta, will be adjust by results (absorbing coeff)
eta = 0.01

############
# need to update @Chih-Tian
h = 1 # related to the bkg function

In [45]:
# Spin (each time)

@njit(parallel = True)
def spin(N, spin, bkg, tags, eta):
  '''
    1) set another array called new playground as nspin to save the new Hierarchy, 
      and update when all units are calculated.
    2) note that no need for bkg since they are indep. for each
    :param N: N sites

  '''
  nspin = spin # this is temp of spin (which depend on spin)
  for i in prange(2, N):
      for j in prange(2, N):
          nspin[i,j] += ((1 / np.sqrt(2*3**2)) * np.exp(-([i,j]/(2 * 3**2))) 
                        * (-(4*spin[i,j] - spin[i,j+1] -spin[i,j-1] - spin[i-1,j] - spin[i+1,j])) 
                    + eta * (np.exp(5)/2 * np.exp(spin[i,j]) 
                    + (1 / (2*np.sqrt(2*3**2))) * np.exp(-(spin[i,j]/(2*3**2)))) * bkg[i,j])
          
          bkg[i,j] += (- eta * (np.exp(5)/2 * np.exp(spin[i,j]) 
                    + (1 / (2*np.sqrt(2*3**2))) * np.exp(-(spin[i,j]/(2*3**2)))) * bkg[i,j])
          # only decay with the time 

  spin = nspin # temp of spin = new spin

  return spin

In [16]:
# Energy (each time)

@njit(parallel = True) # faster than jit if using "parallel"
def energy(spin, tags, J, h):
  energy = tags
  for i in prange(width):
    for j in prange(length):
      s_cen  = spin[i+1, j+1]
      
      s_up   = spin[i  , j+1]
      s_rig  = spin[i+1, j+2]
      s_down = spin[i+2, j+1]
      s_lef  = spin[i+1, j  ]

      energy[i, j] = -J * s_up * s_rig * s_down * s_lef - h * s_cen

  return energy


In [101]:
# Background fucn

@njit(parallel = True)
def bkg(tags, sigma_x, sigma_y):
  # set Background function as BG in size(N,N)
  for i in prange(100):
      for j in prange(100):
          bkg = tags
          bkg[i, j] = (np.exp(-((i - l/2 + 1 )**2 / (2*sigma_x**2) 
                      + (j - l/2 + 1 )**2 / (2*sigma_y**2))))
  return bkg

In [ ]:
# main (loop of time)

def main(N, n, dt, tmax, tags, spin, J, h, sigma_x, sigma_y, eta):
    dt = tmax / n
    t = 0
    while t < tmax:
      
      # update data
      energy(spin, tags, J, h)
      bkg(tags, sigma_x, sigma_y)
      spin(N, spin, bkg, tags, eta)

      # output files
      if (n % io_freq == 0):
        if io_screen:
            '''
            output info files

            '''
            fn = io_folder + "/data_" + io_title + "_" + str(n).zfill(3) + ".txt"
            print(fn)
            output(tags, spin, energy, fn, t)
            output_fig(n, spin, t) # :para arr: arr is a matrix of the grid in time t.
            output_fig(n, energy, t)
            
      # update time
      t += dt
      if t + dt > tmax:
        dt = tmax - t

    return print("Done!")


In [ ]:
######################
# exchange functions #
######################

# loop function (need to use numba to accelerate)

main(N, n, dt, tmax, tags, spin, J, h, sigma_x, sigma_y, eta)